In [9]:
import numpy as np
import matplotlib.pyplot as plt
import random
from copy import deepcopy

def plot(function, x1=-10, x2=10, step=1, x = None):
    x = [x for x in range(x1, x2 + step, step)] if x is None else x
    plt.plot(x, function(np.array(x)))
    plt.show()


def mul_matrix_by_vector(matrix, vector):
    assert len(matrix[0]) == len(vector)
    return [sum(matrix[i][j] * vector[j] for j in range(len(vector))) for i in range(len(matrix))]

def mul_matrix(matrix1, matrix2):
    assert len(matrix1[0]) == len(matrix2)
    return [
        [sum(matrix1[i][k] * matrix2[k][j] for k in range(len(matrix2[0])))
         for j in range(len(matrix2[0]))]
        for i in range(len(matrix1))
    ]

def scalar(vector1, vector2):
    assert len(vector1) == len(vector2)
    return sum(x * y for x, y in zip(vector1, vector2))

def norm(vector):
    return np.sqrt(sum(x**2 for x in vector))

def generate_sim_matrix(n=4, a=-15, b=15, diag = False):
    A = np.zeros(shape=(n, n))
    for i in range(n):
        for j in range(i, n):
            value = random.uniform(a, b)
            A[i][j] = value
            A[j][i] = value
    if diag is not None:
        for i in range(len(A)):
            A[i][i] = abs(A[i][i])
            A[i][i] += diag * sum(abs(A[i][j]) for j in range(n) if j != i)
    return A


def gerchgorin_intervals(A):
    intervals = []
    for i in range(len(A)):
        center = A[i][i]
        radius = sum(abs(A[i][j]) for j in range(len(A)) if j != i)
        intervals.append([center - radius, center + radius])

    intervals.sort(key=lambda x:x[0])
    merged = [intervals[0]]
    for i in range(1, len(intervals)):
        current_interval = intervals[i]
        previous_interval = merged[-1]
        if current_interval[0] <= previous_interval[1]:
            merged[-1] = [previous_interval[0], max(previous_interval[1], current_interval[1])]
        else:
            merged.append(current_interval)

    return merged

def binary_search_roots(intervals, f, delta=1e-3):
    lambdas = []
    for interval in intervals:
        left = interval[0]
        right = interval[1]
        f_left = f(left)
        assert f_left * f(right) < 0
        x = (left + right) / 2
        f_x = f(x)
        while abs(f_x) > delta:
            if f_left * f_x < 0:
                right = x
            else:
                left = x
                f_left = f(left)
            x = (left + right) / 2
            if x == left or x == right:
                break
            f_x = f(x)
        lambdas.append(x)
    return lambdas


def binary_search_intervals(intervals, f, delta=0.05):
    search_intervals = []
    for interval in intervals:
        left = interval[0]
        right = interval[1]
        delta_x = left + delta
        while delta_x <= right:
            f_left = f(left)
            f_delta = f(delta_x)
            if f_left * f_delta < 0:
                search_intervals.append([left, delta_x])
                left = delta_x
                delta_x = left + delta
            else:
                delta_x += delta
    return search_intervals

def check_gerchgorin(intervals, eigs):
    print(eigs)
    print(intervals)
    for eig in eigs:
        interval_found = False
        for interval in intervals:
            if interval[0] <= eig <= interval[1]:
                interval_found = True
                break
        if not interval_found:
            print(f"Eig: {eig} not in gerchgoin circles")
            return
    print("All eigs in gerchgoin circles")

def danilevskiy(A):
    n = len(A)
    B_i = np.diag(np.ones(n))
    for i in range(n):
        if i != (n - 2):
            B_i[n - 2][i] = -1 * A[n - 1][i] / A[n - 1][n - 2]
        else:
            B_i[n - 2][n - 2] =  1 / A[n - 1][n - 2]
    B_i_inv = np.diag(np.ones(n))
    B_i_inv[n - 2] = deepcopy(A[n - 1])
    P = mul_matrix(B_i_inv, A)
    P = mul_matrix(P, B_i)
    B = deepcopy(B_i)
    for i in range(n - 3, -1, -1):
        B_i = np.diag(np.ones(n))
        for j in range(n):
            if j != i:
                B_i[i][j] = -1 * P[i + 1][j] / P[i + 1][i]
            else:
                B_i[i][i] = 1 / P[i + 1][i]
        B_i_inv = np.diag(np.ones(n))
        B_i_inv[i] = deepcopy(P[i + 1])
        P = mul_matrix(B_i_inv, P)
        P = mul_matrix(P, B_i)
        B = mul_matrix(B, B_i)

    return P[0], B

def is_sym_matrix(A):
    for i in range(len(A)):
        for j in range(i, len(A)):
            if A[i][j] != A[j][i]:
                return False
    return True

def eig(A):
    assert len(A) == len(A[0])
    assert is_sym_matrix(A)
    n = len(A)
    intervals = gerchgorin_intervals(A)
    trace = sum(A[i][i] for i in range(n))
    coefs, B = danilevskiy(A)
    coefs = list(map(lambda x: x * -1, coefs))
    f = lambda x: np.polyval([1] + coefs, x)
    search_intervals = binary_search_intervals(intervals, f)
    assert len(search_intervals) == n
    eigs = binary_search_roots(search_intervals, f)
    check_gerchgorin(intervals, eigs)
    print(f"Viet theorem:\nSum = {sum(eigs)}\nTrace = {trace}")
    eig_vectors = []
    for eig in eigs:
        y_vector = [eig ** i for i in range(n - 1, -1, -1)]
        x_vector = np.array(mul_matrix_by_vector(B, y_vector))
        eig_vectors.append(x_vector / norm(x_vector))
    print("Ortnorm eig vectors:")
    print('Norms:')
    for vector in eig_vectors:
        print(norm(vector))
    print("Scalar prods:")
    for i in range(n - 1):
        for j in range(i + 1, n):
            print(scalar(eig_vectors[i], eig_vectors[j]))
    return np.array(eigs), eig_vectors



In [10]:
A = [
    [2.2, 1, 0.5, 2],
    [1, 1.3, 2, 1],
    [0.5, 2, 0.5, 1.6],
    [2, 1, 1.6, 2]
]

print(eig(A))

[-1.4200744628906299, 0.22260742187499594, 1.5453979492187466, 5.652032089233383]
[[-3.5999999999999996, 6.6]]
All eigs in gerchgoin circles
Viet theorem:
Sum = 5.999962997436495
Trace = 6.0
Ortnorm eig vectors:
Norms:
1.0
1.0
1.0
1.0
Scalar prods:
2.2765793934009793e-05
-1.6391075148461387e-05
4.898260302366175e-07
-4.9101734720474743e-05
-3.430284606364964e-08
1.5240101260000083e-06
(array([-1.42007446,  0.22260742,  1.54539795,  5.65203209]), [array([-0.22205713,  0.51591405, -0.75726557,  0.33327494]), array([-0.52195778, -0.45483106,  0.15346446,  0.70507974]), array([ 0.62892696, -0.57257805, -0.48565147,  0.20186111]), array([0.53173734, 0.44619352, 0.40881442, 0.59248419])])


In [11]:
A = generate_sim_matrix(n=10)

print(eig(A))

[-40.09343785685678, -22.009038061133325, -20.194647131268233, 1.5283882488324523, 6.413961853030611, 17.030309694397012, 20.76837969557841, 32.63763662635904, 37.67920477383784, 57.48597264620534]
[[-96.13481111799038, 104.41389214091414]]
All eigs in gerchgoin circles
Viet theorem:
Sum = 91.24673048898237
Trace = 91.24673048897748
Ortnorm eig vectors:
Norms:
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
Scalar prods:
1.7656015538491943e-13
4.0198356796028456e-13
-1.2184697695261093e-13
1.3543333121646128e-13
-1.4915152446448587e-14
1.5543642761794985e-13
-2.0659862709493382e-13
-3.219646771412954e-14
9.306375114981336e-13
4.807924891547799e-13
3.3861802251067274e-15
-1.0935696792557792e-14
5.592748486549226e-14
-3.0284282026560305e-13
-4.322930902134203e-15
-4.153621890878867e-14
1.5015766408055242e-12
-1.3576986757080078e-13
-2.767577833573398e-14
8.64187194027366e-14
-2.370042096605518e-13
-9.087348928904504e-15
-4.721605051383193e-13
-1.011875652712213e-11


In [12]:
A = generate_sim_matrix(n=20)

print(eig(A))

[-49.860995646892945, -48.779574810349104, -44.56357217463621, -26.9629921177264, -22.48569853117835, -14.664411924812587, -12.884529349012606, -8.78984191315281, 1.3554980164691977, 9.375736220138707, 14.084414571327166, 19.759736242605328, 27.779186103639702, 29.973282380999756, 31.42651505297529, 38.79396930241175, 49.4344989193097, 60.277278502959334, 64.3367659208808, 74.69021653432034]
[[-173.67179298425557, 176.62575663348767]]
All eigs in gerchgoin circles
Viet theorem:
Sum = 192.2954813002761
Trace = 192.29548130140648
Ortnorm eig vectors:
Norms:
0.9999999999999999
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999998
0.9999999999999999
0.9999999999999998
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
Scalar prods:
-4.270093546115961e-09
-8.73812887498826e-10
3.843932949720852e-10
-5.035215740883947e-09
-1.8998263830172224e-09
3.1958378374219443e-09
9.107611123604364e-10
1.1794134141540624e-09
2.9641890678111515e-09
2.027002638654851e-09
-3.190652231